In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import swifter
import random
from datetime import datetime
import datetime

In [2]:
def ep_to_day(ep):
    return datetime.datetime.fromtimestamp(ep).weekday()

In [3]:
all_features = ["text_ tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "enaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "enaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "reply_timestamp", "retweet_timestamp",\
                "retweet_with_comment_timestamp", "like_timestamp"]

training = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8", names = all_features, sep="\x01", nrows = 10000)

In [4]:
training['day'] = training.swifter.apply(lambda x: ep_to_day(x['tweet_timestamp']), axis = 1)

In [5]:
training.head()

,text_ tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,enaged_with_user_id,...,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,enaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,day
0,101\t6417\t3410\t3398\t3184\t1909\t56910\t1683...,NaN,D4D1EBDE74F74C5DA529959AF979625C,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,...,27428,600,False,1520948869,False,NaN,NaN,NaN,NaN,6
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,BFB529DAB6D384EB83E899A72AB3830D,Photo,NaN,NaN,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,...,17,77,False,1569692352,True,NaN,NaN,NaN,NaN,3
2,101\t62342\t10858\t54439\t19571\t22480\t7831\t...,NaN,519078C7834E9642508F72A6C2D0F3B7,NaN,788ABAA22F1C4A9D186510C1E3E81F58,892B6E0BC037800B59112CFCB95E3301,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,...,2,29,False,1568107028,False,NaN,NaN,NaN,1.581247e+09,6
3,101\t58955\t10898\t103305\t1901\t16181\t7168\t...,F0F2FBE57F08E7C4326682B5EDA63E3E\tE78674D32346...,52AAE9E33EFAC8C478C57B31A9E31ED1,NaN,C06B84A10A75AE4BF4A102AB45CC2ABF,C0F5328C1D36CD4B34D1808012E18D46,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,...,3,30,False,1568107028,False,NaN,NaN,NaN,NaN,6
4,101\t2435\t5656\t2594\t8279\t8623\t1925\t64126...,NaN,89C1298C55EB3D68E2784F0BFB69E6F8,NaN,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,...,3,30,False,1568107028,False,NaN,NaN,NaN,NaN,3


In [6]:
training['reply_bool'] = training.reply_timestamp.fillna(0)
training.loc[training.reply_bool != 0.0, 'reply_bool'] = 1.0

In [7]:
training['retweet_bool'] = training.retweet_timestamp.fillna(0)
training.loc[training.retweet_bool != 0.0, 'retweet_bool'] = 1.0

In [8]:
training['retweet_with_comment_bool'] = training.retweet_with_comment_timestamp.fillna(0)
training.loc[training.retweet_with_comment_bool != 0.0, 'retweet_with_comment_bool'] = 1.0

In [9]:
training['like_bool'] = training.like_timestamp.fillna(0)
training.loc[training.like_bool != 0.0, 'like_bool'] = 1.0

In [10]:
training['present_links_bool'] = training.present_links.fillna(0)
training.loc[training.present_links_bool != 0, 'present_links_bool'] = 1

In [11]:
training['present_media_bool'] = training.present_media.fillna(0)
training.loc[training.present_media_bool != 0, 'present_media_bool'] = 1

In [12]:
training['present_domains_bool'] = training.present_domains.fillna(0)
training.loc[training.present_domains_bool != 0, 'present_domains_bool'] = 1

In [13]:
(training.reply_bool == 1.0).sum()

280

In [ ]:
[(c, training.columns.get_loc(c)) for c in training.columns]

In [14]:
training_subset = training.iloc[:, [6,7,9,14,17,19,24,25,26,27,28,29,30]]

In [15]:
training_subset.columns

Index(['tweet_type', 'language', 'enaged_with_user_id', 'enaging_user_id',
       'enaging_user_is_verified', 'engagee_follows_engager', 'day',
       'reply_bool', 'retweet_bool', 'retweet_with_comment_bool', 'like_bool',
       'present_links_bool', 'present_media_bool'],
      dtype='object')

In [ ]:
training['day'].nunique()

In [16]:
training_subset.head()

,tweet_type,language,enaged_with_user_id,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,day,reply_bool,retweet_bool,retweet_with_comment_bool,like_bool,present_links_bool,present_media_bool
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,0000006C3074607050F1339DDCB890BB,False,False,6,0.0,0.0,0.0,0.0,0,0
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,000005C520010F8917EEAB6F5B6EC1C4,False,True,3,0.0,0.0,0.0,0.0,0,1
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,False,6,0.0,0.0,0.0,1.0,1,0
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,000006829BEADA9EEA695CF0C334B426,False,False,6,0.0,0.0,0.0,0.0,1,0
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,False,3,0.0,0.0,0.0,0.0,0,0


In [17]:
training_subset.dtypes

tweet_type                    object
language                      object
enaged_with_user_id           object
enaging_user_id               object
enaging_user_is_verified        bool
engagee_follows_engager         bool
day                            int64
reply_bool                   float64
retweet_bool                 float64
retweet_with_comment_bool    float64
like_bool                    float64
present_links_bool            object
present_media_bool            object
dtype: object

In [18]:
training_subset = training_subset.astype(str)

In [19]:
X = training_subset.drop(['reply_bool'], axis = 1)
y = training_subset[['reply_bool']]

In [20]:
X

,tweet_type,language,enaged_with_user_id,enaging_user_id,enaging_user_is_verified,engagee_follows_engager,day,retweet_bool,retweet_with_comment_bool,like_bool,present_links_bool,present_media_bool
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,0000006C3074607050F1339DDCB890BB,False,False,6,0.0,0.0,0.0,0,0
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1F30A85E612A24B426013978ADB7AE46,000005C520010F8917EEAB6F5B6EC1C4,False,True,3,0.0,0.0,0.0,0,1
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,False,6,0.0,0.0,1.0,1,0
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,BEB3C337B266E2C4769065AD64724329,000006829BEADA9EEA695CF0C334B426,False,False,6,0.0,0.0,0.0,1,0
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,False,3,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Retweet,B9175601E87101A984A50F8A62A1C374,4E76346460CF5B0BDAFE15F4F15BD8AB,00231EBA16A507B3DB239D9283C5CFE2,False,True,4,0.0,0.0,1.0,0,1
9996,Quote,22C448FF81263D4BAF2A176145EE9EAD,952B0F59D985675106524A8F93A22F1B,0023208D5F6A8BFBEDD60EA389E34FFD,False,False,0,0.0,0.0,0.0,0,0
9997,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,187AC59639DA9A6F32F7CD118EDD58F7,0023208D5F6A8BFBEDD60EA389E34FFD,False,False,6,0.0,0.0,0.0,1,0
9998,TopLevel,125C57F4FA6D4E110983FB11B52EFD4E,1BC45E70CDD9F4DD7D98A63229189A4E,002320ED0AA42407A190FE2C3F7E336A,False,True,1,0.0,0.0,0.0,0,1


In [21]:
y

,reply_bool
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
9995,0.0
9996,0.0
9997,0.0
9998,0.0


In [22]:
X_numerical = pd.get_dummies(X, sparse = True)

In [23]:
X_numerical

,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,language_022EC308651FACB02794A8147AEE1B78,language_0331BF70E606D62D92C96CE9AD71A7CF,language_06BEAB41D66CCFF329D1ED8BA120A6C2,language_06D61DCBBE938971E1EA0C38BD9B5446,language_125C57F4FA6D4E110983FB11B52EFD4E,language_167115458A0DBDFF7E9C0C53A83BAC9B,language_190BA7DA361BC06BC1D7E824C378064D,...,retweet_bool_0.0,retweet_bool_1.0,retweet_with_comment_bool_0.0,retweet_with_comment_bool_1.0,like_bool_0.0,like_bool_1.0,present_links_bool_0,present_links_bool_1,present_media_bool_0,present_media_bool_1
0,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,1,1,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
4,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,1
9996,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
9997,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,1,1,0
9998,0,0,1,0,0,0,0,1,0,0,...,1,0,1,0,1,0,1,0,0,1


In [ ]:
from collections import defaultdict
d = defaultdict(LabelEncoder)
fit = X.apply(lambda x: d[x.name].fit_transform(x))

In [ ]:
fit

In [ ]:
y

In [ ]:
fs = SelectKBest(score_func=mutual_info_classif, k='all')
fs.fit(fit, y)

In [ ]:
for i in range(len(fs.scores_)):
	print('%s: %f' % (fit.columns[i], fs.scores_[i]))

In [ ]:
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(fit, y)

In [ ]:
for i in range(len(fs.scores_)):
	print('%s: %f' % (fit.columns[i], fs.scores_[i]))

In [ ]:
>>> lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
>>> model = SelectFromModel(lsvc, prefit=True)
>>> X_new = model.transform(X)
>>> X_new.shape

In [24]:
from sklearn.linear_model import LassoCV
import numpy as np
clf = LassoCV().fit(X_numerical, y)
importance = np.abs(clf.coef_)
print(importance)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.00000000e+00 2.71340635e-02 1.40505837e-02 ... 1.53156127e-16
 6.16668237e-03 1.88435359e-19]


In [27]:
[(X_numerical.columns[i], e) for i, e in enumerate(importance) if e != 0]

[('tweet_type_Retweet', 0.02713406349203466),
 ('tweet_type_TopLevel', 0.014050583696060001),
 ('language_06D61DCBBE938971E1EA0C38BD9B5446', 0.0002509368107269597),
 ('language_167115458A0DBDFF7E9C0C53A83BAC9B', 0.009017419890060151),
 ('language_22C448FF81263D4BAF2A176145EE9EAD', 0.004905442935357164),
 ('language_4DC22C3F31C5C43721E6B5815A595ED6', 0.013038028440118227),
 ('language_B9175601E87101A984A50F8A62A1C374', 0.012222753305551008),
 ('language_D3164C7FBCF2565DDF915B1B3AEFB1DC', 0.002447891814855378),
 ('language_ECED8A16BE2A5E8871FD55F4842F16B1', 0.006588571647506259),
 ('enaging_user_id_0006F4CE8EABF29D99DC4A35E1C227A0', 0.012445529322172263),
 ('engagee_follows_engager_False', 0.02498946728168743),
 ('engagee_follows_engager_True', 2.866439087754816e-18),
 ('day_0', 0.002906208687285412),
 ('day_3', 0.0014163490240144045),
 ('day_4', 0.0008513895827196591),
 ('day_5', 0.005130163199617804),
 ('day_6', 0.001948483778492333),
 ('retweet_bool_0.0', 0.000981584487498248),
 ('ret

In [ ]:
tweet_type, language, engagee_follows_engager_True, day, engaged_user_id, engagee_user_id

In [ ]:
fit.columns[2]

In [ ]:
fit.columns[3]

In [ ]:
fit.columns[10]